# Unified Brane-Cosmology — Minimal Likelihood Kit
Load PTA CSV + LISA curve + CMB/BBN priors, fit broken-power-law model with break f_br(lambda) ~ k * lambda**0.25.
Replace templates with real data, run, and export posteriors/figures.


In [ ]:
import numpy as np, pandas as pd, json, matplotlib.pyplot as plt
from scipy.optimize import minimize
pta = pd.read_csv('pta_spectrum_template.csv')
lisa = pd.read_csv('lisa_curve_template.csv')
pri = pd.read_csv('cmb_bbn_priors_template.csv')
DNeff_mu, DNeff_sig = pri.iloc[0]['DeltaNeff_mean'], pri.iloc[0]['DeltaNeff_sigma']

def fbreak_from_lambda(lmbda, k=1e-4):
    return k*(lmbda**0.25)

def model_Omega(f, A1, A2, fb, n1=-2/3, n2=-1):
    return np.where(f<fb, A1*(f/fb)**n1, A2*(f/fb)**n2)

def negloglike(theta):
    loglmbda, A1, A2 = theta
    lmbda = np.exp(loglmbda)
    fb = fbreak_from_lambda(lmbda)
    Om = model_Omega(pta.f_Hz.values, A1, A2, fb)
    chi2 = np.sum(((pta.OmegaGW.values-Om)/pta.sigma_OmegaGW.values)**2)
    # prior on DeltaNeff as a proxy linked to lambda (toy linear map)
    DNeff = 0.15*(lmbda/1.0)**0.0  # placeholder: constant for demo
    chi2 += ((DNeff - DNeff_mu)/DNeff_sig)**2
    return 0.5*chi2

x0 = np.array([0.0, 1e-9, 1e-9])
res = minimize(negloglike, x0, method='Nelder-Mead')
print('best-fit', res.x)

with open('bestfit_demo.json','w') as f:
    json.dump({'log_lambda':float(res.x[0]),'A1':float(res.x[1]),'A2':float(res.x[2])}, f, indent=2)

fb = fbreak_from_lambda(np.exp(res.x[0]))
ff = np.logspace(-9,-2,400)
Om = model_Omega(ff, res.x[1], res.x[2], fb)
plt.figure(); plt.loglog(pta.f_Hz, pta.OmegaGW, 'o'); plt.loglog(ff, Om); plt.xlabel('f [Hz]'); plt.ylabel('Omega_GW'); plt.savefig('pta_fit_demo.png', dpi=180)